<a href="https://colab.research.google.com/github/SteinwayThor/eeg-features-movies/blob/main/EDA_and_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# generic torch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchsummary import summary

# generic ml/stats imports
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy.fftpack as sp
from sklearn.model_selection import train_test_split
from math import pi
from scipy.spatial.distance import pdist, squareform

#torch vision related imports
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets

# data utils
import time
import os
import pickle
import json
from PIL import Image
from google.colab import drive
import bz2
import _pickle as cPickle
import pickle as pl

In [2]:
drive.mount('/content/drive', force_remount=True)

# Load any compressed pickle file
def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = cPickle.load(data)
  return data

Mounted at /content/drive


In [3]:
emotion_annotations_f = '/content/drive/MyDrive/Movie Data/e0018RI/emotion_annotations'  #might need to add Movie Data back in path
emotion_annotations = os.listdir(emotion_annotations_f)

In [7]:
#EDA on emotions json data
diff_emotions = {}
diff_viewers = {}
diff_chapters = {}
for json_file in emotion_annotations:
  split_name = json_file.split("_")
  emotion = split_name[2]  #angry
  viewer = split_name[3]  #viewer127
  chapter = split_name[4][:split_name[4].find(".")]  #chapter01
  diff_emotions[emotion] = diff_emotions.get(emotion,[]).append(json_file)
  diff_viewers[viewer] = diff_viewers.get(viewer,[]).append(json_file)
  diff_chapters[chapter] = diff_chapters.get(chapter,[]).append(json_file)
print(diff_chapters.keys(), diff_emotions.keys(), diff_viewers.keys())

AttributeError: ignored

In [ ]:
f = open(os.path.join(emotion_annotations_f, emotion_annotations[0]))
data = json.load(f)
print(emotion_annotations[0])

json_formatted_str = json.dumps(data, indent=2)
print(json_formatted_str)


In [ ]:
data =
# Create an empty array
#print(data["tracks"])
label_frames = []
for track in data["tracks"]:
    shapes = track["shapes"]
    frame = shapes[0]["frame"]
    attributes = shapes[0]["attributes"]
    if attributes != []:
        emotion = attributes[0]["value"]
        label_frames.append((emotion, frame))

def find_last_frame(data):
    last_frame = 0
    for track in data["tracks"]:
        shapes = track["shapes"][-1]
        frame = shapes["frame"]
        last_frame = frame
    return last_frame

def create_tuple_array(data):
    emotions_frames = []
    current_emotion = "not_angry" #get last emotion from previous dataset
    for i in range(find_last_frame(data)):
        if i in label_frames:
            current_emotion = label_frames[0]
        pair = (current_emotion, i)
        emotions_frames.append(pair)
    return emotions_frames
print(len(create_tuple_array(data)))

In [ ]:
# Load in neural data
# NOTE: Takes a while!
neural_data_folder = '/content/drive/MyDrive/Movie Data/e0018RI/2023-04-02_e0018RI_00'

neural_data_dir = os.listdir(neural_data_folder)

neural_data = {}
channels = []
for f in neural_data_dir:
  channel_name = f[f.rfind('_')+1:-5]
  channels.append(channel_name)
  # channel_data = decompress_pickle(os.path.join(neural_data_folder, f))
  # neural_data[channel_name] = channel_data

print(channels)